# Item 42: Prefer Public Attributes Over Private Ones

In [3]:
# In Python there's only two types of visibility for a class's attributes: private and public
class MyObject:
    def __init__(self):
        self.public_field = 5
        self.__private_field = 10

    def get_private_field(self):
        return self.__private_field

In [4]:
# Public attributes can be accessed by using the dot operator on the object
foo = MyObject()
assert foo.public_field == 5

In [5]:
# Private fields are specified by prefixing an attributes name with a double underscore. They can be
# accessed directly by methods of the containing class
assert foo.get_private_field() == 10

In [6]:
# However, directly accessing private fields from outside the class raises an exception
foo.__private_field

AttributeError: 'MyObject' object has no attribute '__private_field'

In [7]:
# Class methods also have access to private attributes because they are declared within the surrounding
# class block
class MyOtherObject:
    def __init__(self):
        self.__private_field = 71

    @classmethod
    def get_private_field_of_instance(cls, instance):
        return instance.__private_field

bar = MyOtherObject()
assert MyOtherObject.get_private_field_of_instance(bar) == 71

In [8]:
# As we'd expect with private fields, a subclass can't access its parent class's private fields
class MyParentObject:
    def __init__(self):
        self.__private_field = 71

class MyChildObject(MyParentObject):
    def get_private_field(self):
        return self.__private_field

baz = MyChildObject()
baz.get_private_field()

AttributeError: 'MyChildObject' object has no attribute '_MyChildObject__private_field'

The above code fails because of how the Python compiler translates private fields. In this particular case: `_MyChildObject__private_field != _MyParentObject__private_field`, since `__private_field` is only defined in `MyParentObject.__init__`, which means that the private attribute's real name is `_MyParernObject__private_field`.

In [9]:
# We can access private attributes of any class-from a subclass or externally-without asking for permission
assert baz._MyParentObject__private_field == 71

In [10]:
# Private attributes are stored with the names as they appear after the transformation
print(baz.__dict__)

{'_MyParentObject__private_field': 71}


In [14]:
# When it comes to private attributes enforcing strict visibility, Python's philosophy is "we're all
# consenting adults here". We don't need the language to prevent us from doing what we want/need to do.
# New comers to Python use private fields to indicate an internal API that shouldn't be accessed by 
# subclasses or externally
class MyStringClass:
    def __init__(self, value):
        self.__value = value

    def get_value(self):
        return str(self.__value)

foo = MyStringClass(5)
assert foo.get_value() == '5'

In [15]:
# The above code is using the wrong approach. Someone (or you) will eventually want to add new logic and
# subclass our class. These potential subclasses will still need to access the private fields when they
# absolutely need to
class MyIntergerSubclass(MyStringClass):
    def get_value(self):
        return int(self._MyStringClass__value)

foo = MyIntergerSubclass('5')
assert foo.get_value() == 5

In [16]:
# If the class hierarchy changes beneath us, these classes will break because the attribute references are
# no longer valid
class MyBaseClass:
    def __init__(self, value):
        self.__value = value

    def get_value(self):
        return self.__value

class MyStringClass(MyBaseClass):
    def get_value(self):
        return str(super().get_value()) # Updated

class MyIntergerSubclass(MyStringClass):
    def get_value(self):
        return int(self._MyStringClass__value) # Not updated

In [17]:
# __value attribute is now assigned in the MyBaseClass parent class, which causes the private variable
# reference self._MyStringClass__value to break in MyIntegerSubClass
foo = MyIntergerSubclass(5)
foo.get_value()

AttributeError: 'MyIntergerSubclass' object has no attribute '_MyStringClass__value'

In [18]:
# Its better to document each protected field and explain which fields are internal APIs available to 
# subclasses and which should be left alone entirely
class MyStringClass:
    def __init__(self, value):
        # This stores the user-supplied value for the object.
        # It should be coercible to a string. Once assigned in the object
        # it should be treated as immutable.
        self._value = value
        #...

In [19]:
# Only time to consider using private attributes is when we're worried about naming conflicts with subclasses.
class ApiClass:
    def __init__(self):
        self._value = 5

    def get(self):
        return self._value

class Child(ApiClass):
    def __init__(self):
        super().__init__()
        self._value = 'hello' # Conflicts

a = Child()
print(f'{a.get()} and {a._value} should be different')

hello and hello should be different


In [20]:
# To reduce the risk of this naming conflicts from occurring, we can use a private attribute in the parent
# class to ensure that there are no attribute names that overlap with child classes
class ApiClass:
    def __init__(self):
        self.__value = 5 # Double underscore

    def get(self):
        return self.__value  # Double underscore


class Child(ApiClass):
    def __init__(self):
        super().__init__()
        self._value = 'hello'  # OK!


a = Child()
print(f'{a.get()} and {a._value} are different')


5 and hello are different
